## Import Libraries

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

## Check datasets

In [8]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [9]:
builder = tfds.builder(name="horses_or_humans")
builder.info

tfds.core.DatasetInfo(
    name='horses_or_humans',
    version=3.0.0,
    description='A large set of images of horses and humans.',
    homepage='http://laurencemoroney.com/horses-or-humans-dataset',
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=1283,
    splits={
        'test': 256,
        'train': 1027,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {horses_or_humans,
    author = "Laurence Moroney",
    title = "Horses or Humans Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/horses-or-humans-dataset"
    }""",
    redistribution_info=,
)

## Read dataset

In [10]:
train_data, train_info = tfds.load(name="horses_or_humans", split="train", with_info=True)
test_data, test_info = tfds.load(name="horses_or_humans", split='test', with_info=True)

## Transform the images

In [11]:
import tensorflow as tf
IMG_HEIGHT = 200
IMG_WIDTH = 200
train_images = np.array([elem['image'].numpy().astype(np.float32) for elem in train_data])
train_labels = np.array([elem['label'] for elem in train_data])
train_images = tf.image.resize(train_images, (IMG_HEIGHT, IMG_WIDTH))
train_images = train_images/255.0


In [12]:
train_images.shape

TensorShape([1027, 200, 200, 3])

In [13]:
test_images = np.array([elem['image'].numpy().astype(np.float32) for elem in test_data])
test_labels = np.array([elem['label'] for elem in test_data])
test_images = tf.image.resize(test_images, (IMG_HEIGHT, IMG_WIDTH))
test_images = test_images/255.0

## Build model

In [20]:
inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
avgpool = keras.layers.AveragePooling2D(3, 3, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
conv1 = keras.layers.Conv2D(64, 3, activation='relu')
conv2 = keras.layers.Conv2D(32, 5, activation='relu')
conv3 = keras.layers.Conv2D(32, 7, activation='relu')
maxpool = keras.layers.MaxPool2D(3, 2)
dropout = keras.layers.Dropout(0.7)
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(64, activation='relu')
dense2 = keras.layers.Dense(32, activation='relu')
output = keras.layers.Dense(2, activation='softmax')

## Pass Data through the layers

In [21]:
x = avgpool(inputs)
x = conv1(x)
x = conv2(x)
x = conv3(x)
x = maxpool(x)
x = dropout(x)
x = flatten(x)
x = dense1(x)
x = dense2(x)
out = output(x)

## Run the model

In [23]:
model = keras.Model(inputs=inputs, outputs=out, name='horses_or_humans')
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
33/33 [==============================] - 32s 40ms/step - loss: 0.4337 - accuracy: 0.7653
Epoch 2/5
33/33 [==============================] - 1s 36ms/step - loss: 0.2001 - accuracy: 0.9241
Epoch 3/5
33/33 [==============================] - 1s 35ms/step - loss: 0.0888 - accuracy: 0.9688
Epoch 4/5
33/33 [==============================] - 1s 35ms/step - loss: 0.0295 - accuracy: 0.9873
Epoch 5/5
33/33 [==============================] - 1s 35ms/step - loss: 0.0090 - accuracy: 0.9981


In [10]:
## Evaluate the model
model.evaluate(test_images, test_labels)

8/8 [==============================] - 0s 15ms/step - loss: 1.2220 - accuracy: 0.8242


[1.222005844116211, 0.82421875]

## Save an retrain the model

In [11]:
model.save("./horsesOrHumans.h5")

In [12]:
saved_model = keras.models.load_model("./horsesOrHumans.h5")
saved_model.summary()

Model: "horses_or_humans"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 66, 66, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        51232     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 54, 54, 32)        50208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 26, 26, 32)   

In [13]:
saved_model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
33/33 [==============================] - 2s 38ms/step - loss: 0.0513 - accuracy: 0.9805
Epoch 2/5
33/33 [==============================] - 1s 35ms/step - loss: 0.0994 - accuracy: 0.9620
Epoch 3/5
33/33 [==============================] - 1s 35ms/step - loss: 0.0560 - accuracy: 0.9805
Epoch 4/5
33/33 [==============================] - 1s 34ms/step - loss: 0.0402 - accuracy: 0.9873
Epoch 5/5
33/33 [==============================] - 1s 33ms/step - loss: 0.0129 - accuracy: 0.9951


In [15]:
## Evaluate the model
saved_model.evaluate(test_images, test_labels)

8/8 [==============================] - 0s 16ms/step - loss: 1.0996 - accuracy: 0.8633


[1.0996472835540771, 0.86328125]

In [16]:
saved_model.save("./horsesOrHumans.h5")